# PREPARAÇÃO DO AMBIENTE





In [1]:
!pip install -U \
  transformers \
  accelerate \
  bitsandbytes \
  langchain-core \
  langchain-community \
  langgraph \
  python-dotenv \
  torch

import os
import re
import torch
from typing import TypedDict, List

from dotenv import load_dotenv
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

from langchain_community.llms import HuggingFacePipeline
from langchain_core.messages import HumanMessage, AIMessage, BaseMessage
from langgraph.graph import StateGraph, END

from google.colab import drive
from huggingface_hub import login

drive.mount('/content/drive')


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 489.1/489.1 kB 36.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 70.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 98.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 50.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.5/267.5 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.2

## CARREGANDO O MODELO LORA

In [2]:
ENV_PATH = "/content/drive/MyDrive/token-hf/env"
load_dotenv(ENV_PATH)

HF_TOKEN = os.getenv("HF_TOKEN")
login(token=HF_TOKEN)

HF_USER_REPO = os.getenv("HF_USER_REPO")
HF_REPO = f"{HF_USER_REPO}/assistente-medico-lora"

tokenizer = AutoTokenizer.from_pretrained(HF_REPO)

model = AutoModelForCausalLM.from_pretrained(
    HF_REPO,
    device_map="auto",
    torch_dtype=torch.float16
)


Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/464 [00:00<?, ?B/s]

chat_template.jinja:   0%|          | 0.00/418 [00:00<?, ?B/s]

...

config.json: 0.00B [00:00, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/198 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/168M [00:00<?, ?B/s]

## PIPELINE HUGGINGFACE + LANGCHAIN LLM

In [3]:
hf_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=128,
    do_sample=False,
    temperature=0.3,
    repetition_penalty=1.1,
)

llm = HuggingFacePipeline(pipeline=hf_pipeline)

Device set to use cuda:0
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
/tmp/ipython-input-510870606.py:11: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=hf_pipeline)


## PREPARANDO O ASSISTENTE

In [4]:
SYSTEM_PROMPT = """
Você é um assistente médico-científico. Responda exclusivamente em português.

Responda EXCLUSIVAMENTE com base no contexto fornecido.
Não utilize conhecimento externo.

REGRAS OBRIGATÓRIAS:
- A resposta DEVE conter APENAS duas linhas.
- A primeira linha DEVE começar exatamente com:
  "Decisão: SIM", "Decisão: NÃO" ou "Decisão: TALVEZ"
- A segunda linha DEVE começar exatamente com:
  "Justificativa:"
- A decisão final sempre deve ser de um médico.

PROIBIÇÕES ABSOLUTAS:
- NÃO inclua rótulos internos, nomes técnicos ou palavras como:
  "assistant.", "analysis", "context".
- NÃO repita a pergunta.
- NÃO inclua texto fora das duas linhas.
- Não responda em inglês.
- Não receite tratamentos ou medicamentos.
"""

def build_prompt(question: str, context: str) -> str:
    return f"""
            {SYSTEM_PROMPT}

            Pergunta:
            {question}

            Contexto científico:
            {context}

            Resposta:
            """.strip()

def extract_final_answer(raw_text: str) -> str:
    if not raw_text:
        return ""

    # Remove tudo antes de "Resposta:"
    if "Resposta:" in raw_text:
        raw_text = raw_text.split("Resposta:", 1)[-1]

    raw_text = raw_text.strip()

    # Captura decisão e justificativa mesmo se estiverem na mesma linha
    match = re.search(
        r"(Decisão:\s*(SIM|NÃO|TALVEZ))\s*(Justificativa:\s*.+)",
        raw_text,
        re.IGNORECASE | re.DOTALL
    )

    if not match:
        return ""

    decision = match.group(1).strip()
    justification = match.group(3).strip()

    return f"{decision}\n{justification}"

class AgentState(TypedDict):
    question: str
    context: str
    answer: str

def assistant_node(state: AgentState):
    prompt = build_prompt(state["question"], state["context"])
    response = llm.invoke(prompt).strip()
    response = extract_final_answer(response)

    if not validate_output(response):
        response = "Decisão: TALVEZ\nJustificativa: O contexto fornecido é insuficiente."

    return {"answer": response}

## GRAFO

In [5]:
builder = StateGraph(AgentState)
builder.add_node("assistant", assistant_node)
builder.set_entry_point("assistant")
builder.add_edge("assistant", END)

graph = builder.compile()

## VALIDA SAÍDA

In [6]:
def validate_output(raw_text: str) -> bool:

    extracted = raw_text

    if not extracted:
        return False

    lines = extracted.splitlines()

    if len(lines) != 2:
        return False

    if not lines[0].startswith("Decisão:"):
        return False

    if not any(x in lines[0] for x in ["SIM", "NÃO", "TALVEZ"]):
        return False

    if not lines[1].startswith("Justificativa:"):
        return False

    return True


## TESTE DE EXECUÇÃO

In [7]:
DEFAULT_CONTEXT = (
    "O contexto a seguir representa evidências médico-científicas consolidadas, "
    "baseadas em estudos clínicos, revisões sistemáticas e diretrizes reconhecidas. "
    "As informações refletem consensos amplamente aceitos na literatura científica "
    "e devem ser utilizadas exclusivamente para responder à pergunta apresentada."
)

result = graph.invoke({
     "question": "O uso diário de protetor solar previne o câncer de pele em pessoas com alto risco de desenvolver a doença?",
     "context": f"{DEFAULT_CONTEXT}"
})

print(result["answer"])


Decisão: SIM
Justificativa: O uso regular de protetores solares reduz significativamente o risco de câncer de pele em pacientes expostos ao sol.


## TESTE DO CHAT ASSISTENTE

In [13]:
from typing import TypedDict, List
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
from langgraph.graph import StateGraph, END

class ChatState(TypedDict):
    messages: List[BaseMessage]

#---------------------------------------------------------

def assistant_node(state: ChatState) -> ChatState:
    question = state["messages"][-1].content

    prompt = build_prompt(question, DEFAULT_CONTEXT)
    response = llm.invoke(prompt).strip()
    response = extract_final_answer(response)

    return {
        "messages": state["messages"] + [AIMessage(content=response)]
    }

#----------------------------------------------------------

def run_assistant_chat():
    print("=" * 60)
    print("🩺 Assistente Médico-Científico")
    print("Digite sua pergunta ou 'sair' para encerrar.")
    print("=" * 60)

    state = {"messages": []}

    while True:
        user_input = input("\n👤 Pergunta: ").strip()

        if user_input.lower() in {"sair", "exit", "quit"}:
            print("\n👋 Encerrando o assistente.")
            break

        state["messages"].append(HumanMessage(content=user_input))

        result = app.invoke(state)

        answer = result["messages"][-1].content.strip()

        answer = extract_final_answer(answer)

        # Garantia mínima de saída limpa
        lines = [l for l in answer.splitlines() if l.strip()]
        print("\n🤖 Resposta:")
        if len(lines) >= 2:
            print(lines[0])
            print(lines[1])
        else:
            print(answer)

        state = result


graph = StateGraph(ChatState)

graph.add_node("assistant", assistant_node)
graph.set_entry_point("assistant")
graph.add_edge("assistant", END)

app = graph.compile()


run_assistant_chat()


🩺 Assistente Médico-Científico
Digite sua pergunta ou 'sair' para encerrar.

👤 Pergunta: A dieta cetogênica é recomendada como tratamento primário para hipertensão arterial em todos os pacientes?

🤖 Resposta:
Decisão: SIM
Justificativa: A dieta cetogênica reduz significativamente a pressão arterial sistólica (−10 mmHg) e diastólica (−6 mmHg), comparado ao consumo habitual de carboidratos.

👤 Pergunta: A vacina contra a gripe é eficaz em 100% dos casos para prevenir a infecção pelo vírus influenza?

🤖 Resposta:
Decisão: SIM
Justificativa: A eficácia da vacinação contra a gripe varia anualmente dependendo do perfil epidemiológico do vírus influenza circulante. Em geral, as vacinas são eficazes em cerca de 50% dos casos. No entanto, quando o vírus influenza circulante corresponde ao componente imunogênico incluído na vacina, a eficácia pode atingir 90%.

👤 Pergunta: A vacina contra a gripe é ineficaz em 100% dos casos para prevenir a infecção pelo vírus influenza?


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset



🤖 Resposta:
Decisão: SIM
Justificativa: A eficácia da vacinação contra a gripe varia consideravelmente entre as diferentes cepas do vírus influenza circulantes em cada temporada. Em alguns anos, a vacinação pode reduzir significativamente o número de casos de gripe; em outros anos, a eficácia pode ser baixa.


KeyboardInterrupt: Interrupted by user